# how I created the random data

Excel file was created manually in Excel
I created two sheets, each with different random integers in the first column

Then I mixed entries from the second sheet with the values in the first sheet.
https://apple.stackexchange.com/questions/142860/install-shuf-on-os-x

`jot -r "$(wc -l FILE)" 1 |
paste - FILE |
sort -n |
cut -f 2- |
head -n 10`

# Week 5 Assignment description

You get an Excel spreadsheet that has two worksheets, "patient info" and "records"

Each worksheet has data and visualizations and formulas
Each row in each worksheet represents information associated with a single person.. 
Worksheet "patient info" has a "patient id" column; 
Worksheet "records" has a "p_id" column
The "patient id" values in "patient info" tab are the same variable as "p_id" in "records". Some people have information on both sheets, while other people have information only on one sheet.
Use openpyxl to copy information about patients from "records" to "patient info"​. Submit a notebook that reads the Excel spreadsheet and produces a separate spreadsheet with the following modifications:

1. For each patient identified by "p_id" listed in the "records" sheet that don't exist in "patient info," create a new row in "patient info". The row from "records" should be copied to the row in "patient info."
2. For each patient listed in the "records" sheet where "p_id" appears "patient info" under "patient id," copy the data in the row from records to "patient info" sheet
3. Make no changes to the visualizations that exist in each worksheet
4. Make no changes to the data in the "records"sheet
5. Write your changes to a new .xlsx file (don't overwrite the original)

Observations:

1. "patient info" worksheet will have new columns (because those new columns exist in the "records" worksheet
2. "patient info" worksheet will have new rows (one new row per patient)
3. There will be empty cells in "patient info" worksheet
4. Use a programmatic (rather than manual) approach to identify which patients appear in both worksheets
5. Some cells in both worksheets contain formulas. Copy only values from "records" to "patient info"

# Install openpyxl

Source : https://openpyxl.readthedocs.io/en/stable/

In [1]:
# using pip to install openpyxl in the present instance
!pip install openpyxl

In [2]:
# importing the openpyxsl module 
import openpyxl as xsl

# load data
load_workbook() method load the excel sheet to a temporary workbook. This workbook in in python instance. 

In [3]:
# The excel file is loaded to wb. 
# This wb is later modified and saved as file name

wb = xsl.load_workbook('week_05_homework_XLSX_openpyxl.xlsx')

In [4]:
# Printing the sheet names of the excel.
# Sheetnames attribute is used to print all the sheet names of the Excel

wb.sheetnames

['patient info', 'records']

The sheets are now loaded to a workbook instance. again, these are only in python instance. 

In [5]:
# Loading the sheet data into work book instance.
# patientinfo and records holds the sheet value of the excel. 

patientinfo = wb['patient info']
records = wb['records']

openpyxl enables to count maximum rows and columns. Max_row and max_colum are used to count the rows and column of patient_info sheet

In [6]:
# Counting the maximum rows and Columns
# here patientinfoRowCount is added to 1 to get the next vacant row index from where the data should be appended. 

patientinfoRowCount = patientinfo.max_row + 1
patientinfoColumnCount = patientinfo.max_column

Since the values are being coped from records sheet to patient_info shhet, to make the excel sheet more readable, the column names (headings) are also appended to the patien_info sheet after the maxmim column count index

In [7]:
# This loop iterates among the columns of the records Sheet
# Since I am appending the column headings, the row index will always be 1 
# While the column index of patient info starts from 7(> Column count ) to maximum column count of records sheets
# The column index of records sheet will start from 2, since I am not appending p_id heading
for j in range(1,records.max_column):
    patientinfo.cell(row = 1, column = j + patientinfoColumnCount).value =  records.cell(row = 1, column = j + 1).value 

Since the Patent_id and p_id are being compared, I copied all the pateint_id values and p_id values to seperate lists

In [8]:
# This code will copy all the patent_id values to the list. 
# The range of excel starts from 1 unlike list which is 0 
# to remove the headers, the range is started from 2 to maxmium row count

patientinfoList = []
for i in range(2,patientinfo.max_row):
    patientinfoList.append(patientinfo.cell(row = i, column = 1).value)

In [9]:
# This code will copy all the p_id values to the list. 
# The range of excel starts from 1 unlike list which is 0 
# to remove the headers, the range is started from 2 to maxmium row count

recordsList = [] 
for i in range(1,records.max_row):
    recordsList.append(records.cell(row = i, column = 1).value)

Now that, I have all the row values in seperate list, It's time to compare. In order to get the p_id 
from records that are also present in the patient_info
I will have loop through records sheet and match them with the patient_info list.

In [10]:
# Declaring am empty list which will hold the matching values from both sheets

appendList = []
for j in range(2,records.max_row):
    if(records.cell(row = j, column = 1).value in patientinfoList): #This condition is used to match each cell of the p_id from records with the patient list
            appendList.append(records.cell(row = j, column = 1).value) #if match is found then it is appended to the declared list

This logic is used to append the column data of the p_id from records sheet against patient_id in the patient info sheet

In [11]:
# This logic contains multiple for loops. it will iterate through rows of patientinfo 
# if the match is found in the appenlist the it will iterate through the columns of records Sheet and it will append the column values in the patientinfo sheet. 
# the new colum index will be dynamic starting aftet the existing column indexes. 
# rowIndexOfRecords hold the index value of the matched p_id value from records sheets
# This index is used to retrive the columns where the row index being rowIndexOfRecords constant and column index being dynamic

for i in range(1,patientinfo.max_row):
    if(patientinfo.cell(row=i, column = 1).value in appendList): 
        rowIndexOfRecords = recordsList.index(patientinfo.cell(row=i, column = 1).value)
        for j in range(1,records.max_column ):
            patientinfo.cell(row = i , column = j+patientinfoColumnCount).value = records.cell(row = rowIndexOfRecords + 1, column = j+1).value  

The below code is used to append all the the unmatched p_id values with column data from records  sheet to patient info sheet

In [12]:
# This list hold the tuples of records, which have the same pid and patient_id. 
# This list is later used to get the unmatches values 
sameTuplesList = []
for rows in records.iter_rows():
    for value in patientinfo.iter_rows():
        if(rows[0].value == value[0].value):
            sameTuplesList.append(rows)

In [13]:
# This list is used to hold the tuples of unmatched p_id values of records sheet 
# This loop rocrds sheet and checks the data aginst the matched data i.e. sameTuplesList list. If not present it copies the row value to this list 
differentList = []
for row in records.iter_rows():
    if(row not in sameTuplesList):
        differentList.append(row)

In [14]:
# Since the data  comes with the heading, this logic is used to pop it out. 
# since the values are stored in the list, the headings are stored in the 0th index. 
heading = differentList.pop(0)

In [15]:
# This logic is used to append the row values at the end of patient_info sheet. 
for value in differentList:
    for j in range(2,records.max_column + 1):
        patientinfo.cell(row = patientinfoRowCount, column = 1).value = value[0].value
        patientinfo.cell(row = patientinfoRowCount, column = j + 5).value = value[j-1].value
    patientinfoRowCount = patientinfoRowCount + 1

In [2]:
# saving the working to a new file
wb.save('week_05_result.xlsx')

NameError: name 'wb' is not defined